First, please install the packages needed for the conversational AI:

In [ ]:
! pip install evaluate
! pip install torch
! pip install transformers
### maybe you will need to install some additional packages

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 977.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. T

# 1 Chatbots and Conversational question answering


```
# This is formatted as code
```

### 1.1 You want to build a dense passage retriever for the question-answering task. Now you have the labeled question-answering dataset, and each sample contains the question, answer, and related passages. From the lecture, you know that we can build the negative passages by using any random passage from the dataset. Do you have any other idea to build the negative passages such that fine-tuned model gets better performance?
There are two other methods to choose the negative passages from the dense passage retriever paper.
BM25: top passages returned by BM25 which don’t contain the answer but match most question tokens;
Gold: positive passages paired with other questions which appear in the training set.

### 1.2 After building the training dataset for the dense passage retriever, we have B questions in a mini-batch, and each one is associated with a relevant passage. Do you have any idea to make the training effective?
Let Q and P be the (B×d) matrix of question and passage embeddings in a batch of size B. S = QPT is a (B × B) matrix of similarity scores, where each row of which corresponds to a question, paired with B passages.

### 1.3 From the lecture, we know that the system should consider the dialogue history for the conversational question-answering task. Now we have the following conversational question-answering task and want to use a GPT2 model to generate the correct answer. How will you design the input for the GPT2 model?

Jessica went to sit in her rocking chair. Today was her birthday and she was turning 80. Her granddaughter Annie was coming over in the afternoon and Jessica was very excited to see her. Her daughter Melanie and Melanie’s husband Josh were coming as well. Jessica had …

Q1: Who had a birthday?
A1: Jessica
Q2: How old would she be?
A2: 80
Q3: Did she plan to have any visitors?
A3: Yes
Q4: How many?
A4: Three
Q5: Who?

Answer: [Context] Jessica went to sit in her rocking chair. Today was her birthday and she was turning 80. Her granddaughter Annie was coming over in the afternoon and Jessica was very excited to see her. Her daughter Melanie and Melanie’s husband Josh were coming as well. Jessica had … [Question]Who had a birthday? [Answer]Jessica [Question]How old would she be? [Answer]80 [Question]Did she plan to have any visitors? [Answer]Yes [Question]How many? [Answer] Three[Question] Who?

## 1.4 Dense passage retriever from Huggingface

In [ ]:
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer

####################################################################
    # TODO choose the suitable question model and tokenizer for question from huggingface
####################################################################
tokenizer_question = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
model_question = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
####################################################################
    # TODO Tokenize the question and feed the tokenizer
    # output into the question model.
    # question "what is ChatGPT ?"
####################################################################
input_ids = tokenizer_question("what is ChatGPT ?", return_tensors="pt")["input_ids"]
embeddings_q = model_question(input_ids).pooler_output
print(embeddings_q)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/493 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[-8.3379e-02, -3.1729e-01,  2.1917e-01,  2.9387e-01,  7.3082e-02,
          3.6832e-02,  1.3859e-01,  4.2937e-01, -2.2792e-01, -1.1773e-01,
         -2.1555e-01,  1.0881e-02, -7.2254e-01,  4.8839e-02, -1.6912e-02,
         -3.1558e-01,  2.6363e-01, -1.2033e-01, -4.6281e-01, -4.1088e-01,
          2.8026e-04, -1.6217e-01, -5.4613e-01, -3.4272e-01,  4.0239e-01,
          4.1672e-01, -2.6467e-01,  2.5873e-01, -4.0853e-01, -2.2410e-01,
          6.0410e-02, -3.4260e-01,  1.8166e-01, -2.0097e-01, -3.6905e-02,
         -2.7795e-02, -2.5723e-01,  1.0948e-01, -1.1370e-01, -5.0613e-02,
         -1.7084e-01,  1.7614e-02,  1.2492e-01,  1.3045e-01, -2.0563e-01,
          1.8169e-01, -2.7088e-01,  3.1271e-01, -3.8192e-01,  8.1030e-02,
         -4.4155e-01,  7.8729e-01,  3.6296e-01,  3.1146e-01,  6.8964e-02,
         -2.4018e-01,  3.6706e-01, -3.7625e-01, -2.9906e-01,  5.0508e-02,
         -6.7474e-02, -2.0888e-01,  1.1135e-01,  2.0920e-01, -1.8659e-01,
          1.9057e-01, -9.6297e-02,  8.

In [ ]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
####################################################################
    # TODO choose the suitable context model and tokenizer for context from huggingface
####################################################################
tokenizer_context = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
model_context = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
####################################################################
    # TODO Tokenize the passage 1 and feed the tokenizer
    # output into the context model.
    # passage 1 "ChatGPT is an artificial intelligence chatbot developed by OpenAI."
####################################################################
input_ids = tokenizer_context("ChatGPT is an artificial intelligence chatbot developed by OpenAI.", return_tensors="pt")["input_ids"]
embeddings_p1 = model_context(input_ids).pooler_output
print(embeddings_p1.shape)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/492 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([1, 768])


In [ ]:
import torch
####################################################################
    # TODO calculate the score between the user query and the passage 1
####################################################################
score1 = torch.mm(embeddings_q, embeddings_p1.transpose(0, 1))[0].cpu().tolist()
print('The score between the user query and the passage 1 is: ', score1)

The score between the user query and the passage 1 is:  [94.91858673095703]


In [ ]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
####################################################################
    # TODO Tokenize the passage 2 and feed the tokenizer
    # output into the context model.
    # passage 2 "A chatbot amis to simulate human communication."
####################################################################
input_ids = tokenizer_context("A chatbot amis to simulate human communication.", return_tensors="pt")["input_ids"]
embeddings_p2 = model_context(input_ids).pooler_output
print(embeddings_p2.shape)

torch.Size([1, 768])


In [ ]:
score1 = torch.mm(embeddings_q, embeddings_p2.transpose(0, 1))[0].cpu().tolist()
print('The score between the user query and the passage 2 is: ', score1)
####################################################################
    # TODO calculate the score between the user query and the passage 2
####################################################################

The score between the user query and the passage 2 is:  [70.62039947509766]


## 1.5 SingleEncoder

We can also only use the query encoder to do the retrieval. What is the difference?

In [ ]:
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
####################################################################
    # TODO choose the suitable question model and tokenizer for question from huggingface
####################################################################
tokenizer_question = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
model_question = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
####################################################################
    # TODO Tokenize the question and feed the tokenizer
    # output into the question model.
    # question "what is ChatGPT ?"
####################################################################
input_ids = tokenizer_question("what is ChatGPT ?", return_tensors="pt")["input_ids"]
embeddings_q = model_question(input_ids).pooler_output
print(embeddings_q.shape)

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([1, 768])


In [ ]:
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
####################################################################
    # TODO Tokenize the passage 1 and feed the tokenizer
    # output into the question model.
    # passage 1 "ChatGPT is an artificial intelligence chatbot developed by OpenAI."
####################################################################

input_ids = tokenizer_question("ChatGPT is an artificial intelligence chatbot developed by OpenAI.", return_tensors="pt")["input_ids"]
embeddings_p1 = model_question(input_ids).pooler_output
print(embeddings_p1.shape)

torch.Size([1, 768])


In [ ]:
import torch
####################################################################
    # TODO calculate the score between the user query and the passage 1
####################################################################
score1 = torch.mm(embeddings_q, embeddings_p1.transpose(0, 1))[0].cpu().tolist()
print('The score between the user query and the passage 1 is: ', score1)

The score between the user query and the passage 1 is:  [89.18318176269531]


In [ ]:
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer

input_ids = tokenizer_question("A chatbot amis to simulate human communication.", return_tensors="pt")["input_ids"]
embeddings_p2 = model_question(input_ids).pooler_output
####################################################################
    # TODO Tokenize the passage 2 and feed the tokenizer
    # output into the question model.
    # passage 2 "A chatbot amis to simulate human communication."
####################################################################
print(embeddings_p2.shape)

torch.Size([1, 768])


In [ ]:
import torch
####################################################################
    # TODO calculate the score between the user query and the passage 2
####################################################################
score2 = torch.mm(embeddings_q, embeddings_p2.transpose(0, 1))[0].cpu().tolist()
print('The score between the user query and the passage 2 is: ', score2)

The score between the user query and the passage 2 is:  [78.11785888671875]


# 2 RAGs

## 2.1 Questions

### 2.1.1 What action is necessary to incoorporate new data into a RAG system?

The new data needs to be added into the vector store. This essentially means chunking and embedding of the new data.

### 2.1.2 Is there any benefit in finetuning the retrieval/embedding and synthesis LMs within the RAG pipeline? Please justify your answer within about 1 to 4 sentences.

Yes. While the components don't need to learn any knowledge, they still need to be able to understand the language of the provided data as good as possible. This allows optimal selection and interpretation of the provided data.

### 2.1.3 Which requires more effort: to exchange the retrieval/embedding model or to exchange the synthesis model? Please justify your answer within about 2 to 6 sentences.

Usually exchanging the embedding model requires a lot of compute time as the whole data base needs new embeddings. Old embeddings can not be reused, as the distance relation only holds within the embedding space of one LM.
Of course adjusting a new synthesis model can also take a lot of effort, e.g. when extensive propmt optimization is necessary.

### 2.1.4 What practical challenge might be caused by using a semantic splitter instead of a text-length based one?

Standard semantic chunking does not ensure a fixed upperbound for chunk length. This might cause trouble facing limited context lenght of the synthesis LM.

## 2.2 Basic langchain RAG pipeline



One convinient option to implement a RAG pipeline is langchain. Implement a simple pipeline following this tutorial: https://python.langchain.com/v0.2/docs/tutorials/rag/. Use the provided cells.

In [ ]:
### Required installs:
!pip install langchain langchain-community langchain-core langchain_experimental langchain_chroma langchainhub
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
### Load the content of the webpage: "https://lilianweng.github.io/posts/2023-06-23-agent/"
### Solution:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

In [ ]:
### Inspect the data (to get familiar; usefull in case of bugs etc.)
print(len(docs))
print(len(docs[0].page_content))
print(docs[0].page_content[:500])

1
43131


      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


In [ ]:
### To be able to use a small local model, with a small context window, we use a small chunk size
chunk_size=100
chunk_overlap=10

In [ ]:
### Split the content using the RecursiveCharacterTextSplitter
### Solution
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap, add_start_index=True
)
splits = text_splitter.split_documents(docs)

In [ ]:
### Load a local embedding model.
from langchain.embeddings import HuggingFaceEmbeddings

embedding_lm = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L6-v2',     # Provide the pre-trained model's path
    model_kwargs={'device':'cpu'}, # Pass the model configuration options
    encode_kwargs={'normalize_embeddings': True} # Pass the encoding options
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
### Create a vectore database using chroma and the following configuration.
searchtype = "similarity"
k = 3
### Solution
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_lm)
retriever = vectorstore.as_retriever(search_type=searchtype, search_kwargs={"k": k})

In [ ]:
### Test your retriever for basic functionality.
retrieved_docs = retriever.invoke("What are the approaches to Task Decomposition?")
print(retrieved_docs[0].page_content)

Task Decomposition#


In [ ]:
### Load a local generation model, to be used for the synthesis step.
### Feel free to replace this by something more powerfull.
### Try an api in case you got access,
### or a bigger model, but mind the resource needs.
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

lm = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 300},
)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
### Load a promt.
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [ ]:
### Set up the RAG chain.
### Solution
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs) + "\n\n"

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | lm
    | StrOutputParser()
)

In [ ]:
### Run the chain for one question
for chunk in rag_chain.stream("What is Task Decomposition?"):
    print(chunk, end="", flush=True)

Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: What is Task Decomposition? 
Context: Task Decomposition#

Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals,

 
Answer: What are we doing?: A program to decompose large tasks into smaller tasks is performed by LLM(1). "Solution": "If we did a task one last time; this one has less

Task complexity than

"Solution" is to say that this task should be less task in scale than its

"Solution" is to say that "Task Decomposition #: What are we doing".

Now, a program (i.e., Task Decomposition#) works like this (1): "Create a small task in the program's global scope by "toting" to it on a list.

(Ex: We

# 3 Adapter-based Knowledge Enhancement

You want to infuse a LM with structured knowledge, but have limited computing resources to do so. You chose to make use of adapters as a solution. In the following, get familiar with the high-level functions of the adapters library. Use the Adapter Hub documentation for help on the inner workings of the code. This part of the notebook is a modified version of the AdapterHub QuickStart tutorial (see references).

Lets install the Adapter-Hub package before we start (you might have to restart the runtime before starting so you dont have any issues with libraries or models loaded earlier).

In [ ]:
!pip install adapters

### 3.1 Adapter Basics

The following example shows how to enhance a basic pre-trained Transformer model with adapters. Our goal here is to predict the sentiment of a given sentence.

We use GPT2 in this example. We first load a pre-trained tokenizer to encode the input sentence and a pre-trained checkpoint from Hugging Face’s Model Hub using the GPT2AdapterModel class:

In [ ]:
import os
import torch

from transformers import AutoTokenizer
from adapters import GPT2AdapterModel, AutoAdapterModel


tokenizer = AutoTokenizer.from_pretrained("gpt2")

# An input sentence
sentence = "What a lovely time to learn about such facinating language models"

# Tokenize the input sentence and create a PyTorch input tensor
input_data = tokenizer(sentence, return_tensors="pt")

# Load pre-trained GPT2 model from Hugging Face Hub
# The `(GPT2)AdapterModel` class is specifically designed for working with adapters
# It can be used with different prediction heads
model = GPT2AdapterModel.from_pretrained('gpt2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


After loading the model, we add a pre-trained task adapter from AdapterHub that is relevant to our task. For sentiment classification, we use an adapter trained on the SST-2 dataset. The task prediction head loaded with the adapter then provides a class label for our sentence.

In [ ]:
# Load pre-trained Pfeiffer task adapter from Adapter Hub
# This method call will also load a pre-trained classification head for the adapter task
adapter_name = model.load_adapter("sentiment/sst-2@ukp", config='pfeiffer')

# Activate the adapter we just loaded, so that it is used in every forward pass
model.set_active_adapters(adapter_name)

# Predict output tensor
outputs = model(**input_data)

# Retrieve the predicted class label (1 = positive sentiment)
predicted = torch.argmax(outputs[0]).item()
assert predicted == 1
print(predicted)

1


To save our pre-trained model and adapters, we can easily store and reload them as follows:

In [ ]:
# For the sake of this demonstration an example path for loading and storing is given below
example_path = os.path.join(os.getcwd(), "adapter-quickstart")

# Save model
model.save_pretrained(example_path)
# Save adapter
model.save_adapter(example_path, adapter_name)

# Load model, similar to Hugging Face's AutoModel class,
# you can also use AutoAdapterModel instead of GPT2AdapterModel
model = AutoAdapterModel.from_pretrained(example_path)
model.load_adapter(example_path)

Removed shared tensor {'heads.default.0.weight'} while saving. This should be OK, but check by verifying that you don't receive any warning while reloading


'sst2'

Similar to how the weights of the full model are saved, the save_adapter() will create a file for saving the adapter weights and a file for saving the adapter configuration in the specified directory.

Finally, if we have finished working with adapters, we can restore the base Transformer to its original form by deactivating and deleting the adapter:

In [ ]:
# Deactivate all adapters
model.set_active_adapters(None)
# Delete the added adapter
model.delete_adapter(adapter_name)


### 3.2 Adapter Poetry

Now combine what you have learned to teach a GPT2 model how to generate poems using adapters! To get you started, we give you an example for generating text with GPT2 without an adapter loaded:

In [ ]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained("gpt2")

prompt = "Roses are red"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

gen_tokens = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.9,
    max_length=100,
    pad_token_id=tokenizer.eos_token_id
)

gen_text = tokenizer.batch_decode(gen_tokens)[0]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# The generated text should be typical GPT2 gibberish without any poetic structure.
print(gen_text)

Roses are red, but otherwise they are white:

You would think, "This is just part of our process." But of course, in a world where you are a few hundred years ahead of us, that's not quite the case. These guys aren't just working for the dollar, they are doing the same thing for the world. Our planet has already been colonized by a couple hundred years. They're setting the world before us. Some of these early explorers weren't very


In [ ]:
from adapters import AdapterConfig

prompt = "Roses are red"

model = AutoAdapterModel.from_pretrained("gpt2")
config = AdapterConfig.load("pfeiffer", non_linearity="relu", reduction_factor=16)
adapter_name = model.load_adapter("lm/poem@ukp", config=config)

model.set_active_adapters(adapter_name)

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

gen_tokens = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.9,
    max_length=100,
    pad_token_id=tokenizer.eos_token_id
)

gen_text = tokenizer.batch_decode(gen_tokens)[0]


(…)b/v2/poem/gpt2/gpt2_en_poem_pfeiffer.zip:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

In [ ]:
# Now the same prompt should give a generation with clear poetic structure and a shakespearesque touch.
print(gen_text)

Roses are red, but not white,so far as to protect a faint green sun.

and here, in the heart of the sea, are the tears that keep her away,as to the bitter, as the bitter of winter.--the last sorrows of your day, if you wish.--the last, if you wish.--and then, while on the shore of the sea, it is this...and that...and then, when the last of the cold winds


## 3.3 Adapter Training
To not extend this tutorial too much and not let you wait through an hour of training, we only refer you to https://docs.adapterhub.ml/training.html for a tutorial on Adapter Training. The next code cell gets you started on the example projects provided by AdapterHub, but there will be no exam question on any code for adapter *training*.

In [ ]:
# tmp installs and imports from previous cells
!git clone https://github.com/adapter-hub/adapters
!cd adapters
!pip install .
!pip install -r ./examples/pytorch/<your_examples_folder>/requirements.txt

## 3.4 Open Questions

### 3.4.1
**What are the advatages and disadvantages of adapter-based approaches when compared to standard fine tuning?**

Main Advantages:
- Higher computational effiiency (only 1-8 percent of updated parameters when compared to full model fine tuning)
- Mitigates catastrophic forgetting (parameters that hold the information from
pre-training are frozen)

Main Disadvatages:

- Slightly hinger inference time
- Possibly worse overall performance (depending on downstream task and domain)

### 3.4.2
**You want to inject structured knowledge from a classical head-relation-tail triplets knowledge graph into your LLM. Since you have limited resources, you want to leverage  adapters in your work. What training objectives could you use to train the adapters?**

- Entity Prediciton
- Link Prediciton
- Transform triplets to natural language and perform standard language modeling or any other related training objective

### 3.4.3
**Due to the large number of entities in your chosen KG, computing an exact softmax over all entities is very expensive for training and predicting. What could be a potential solution to this problem? What is an appropriate algorithm that you could use in the implementation?**

Partition the graph into k sub-graphs that adapters can be trained on separately. Possible algorithm: METIS



## Citations
Karpukhin, V., Oguz, B., Min, S., Lewis, P., Wu, L., Edunov, S., ... & Yih, W. T. (2020, November). Dense Passage Retrieval for Open-Domain Question Answering. In Proceedings of the 2020 Conference on Empirical Methods in Natural Language Processing (EMNLP) (pp. 6769-6781).

Huggingface https://huggingface.co/ DPR model, ect.

ChatGPT Introducing ChatGPT https://openai.com/blog/chatgpt

Tang, F., Zeng, L., Wang, F., & Zhou, J. (2021). Persona authentication through generative dialogue. arXiv preprint arXiv:2110.12949.

Radford, Alec, et al. "Language models are unsupervised multitask learners." OpenAI Blog 1.8 (2019): 9.

Thomas Wolf et al. "Transfertransfo: A transfer learning approach for neural network based conversational agents." arXiv preprint328arXiv:1901.08149, 2019


Pfeiffer, J., Rücklé, A., Poth, C., et al. (2020). AdapterHub: A Framework for Adapting Transformers. https://arxiv.org/abs/2007.07779

Meng, Z., Liu, F., Clark, et al. (2021). Mixture-of-Partitions: Infusing Large Biomedical Knowledge Graphs into BERT. Conference on Empirical Methods in Natural Language Processing. https://arxiv.org/abs/2109.04810

Poth, C.A., Sterz, H., Paul, I., et al. (2023). Adapters: A Unified Library for Parameter-Efficient and Modular Transfer Learning. Conference on Empirical Methods in Natural Language Processing. https://arxiv.org/abs/2311.11077


